# zCFD mesh guide
---

This notebook provides an interactive description of the zCFD mesh format. 

zCFD meshes are unstructured, cell centred meshes, stored in hdf5 format.

To interact with hdf5 files in python, import the h5py module- note this comes shipped within the zCFD virtual environment. A mesh can be loaded with the method `File`

In [57]:
import h5py
import numpy as np

fname = '../data/MDO_125K.h5'
h5file = h5py.File(fname,"r")

print('Mesh file: {} loaded successfully'.format(fname))

Mesh file: ../data/MDO_125K.h5 loaded successfully


## Groups
---
At the highest level the mesh has a group called 'mesh'. Groups in hdf5 format serve as 'directory like' to provide structure to the dataset, and can have aspects of python dictionary convention such as `keys`, `values`, and iteration support. 

In [58]:
groups = list(h5file.keys())
print(groups)

['mesh']


The mesh group contains all the mesh data, and can be extracted using the `get` method.

In [59]:
mesh = h5file.get('mesh')

print('Mesh group successfully loaded')

Mesh group successfully loaded


## Attributes
---

The mesh group has 2 int64_t formatted attributes- `numCells`, and `numFaces`. 

In [60]:
attributes = list(mesh.attrs.keys())
print(attributes)

['numCells', 'numFaces']


Or if using dictionary notation-

In [61]:
attributes = list(h5file['mesh'].attrs.keys())
print(attributes)

['numCells', 'numFaces']


As before these attributes can be accessed using the `get` method:

In [62]:
numCells = int(mesh.attrs.get('numCells'))
numFaces = int(mesh.attrs.get('numFaces'))

print(numCells, numFaces)

124800 382720


Alternatively the extraction can be automated using the `items` method:

In [63]:
attributes = dict(mesh.attrs.items())

print('Attributes: {}'.format(attributes))

# Change to int format using dictionary comprehension
attributes = {item:int(attributes[item]) for item in attributes}

print('Attributes: {}'.format(attributes))

Attributes: {'numCells': array([[124800]], dtype=int32), 'numFaces': array([[382720]], dtype=int32)}
Attributes: {'numCells': 124800, 'numFaces': 382720}


## Datasets
---
Datasets are analogous to arrays in hdf5 format. They are where the physical data is stored in the zCFD mesh. The 6 required datasets are:
* faceBC
* faceCell
* faceInfo
* faceNodes
* faceType
* nodeVertex

*note from here on the mesh group is used to explore the data structure, but it can be exchanged for `h5file['mesh']` for equivalent results*

In [64]:
datasets = list(mesh.keys())

print(datasets)

['cellFace', 'faceBC', 'faceCell', 'faceInfo', 'faceNodes', 'faceType', 'nodeVertex']


Similarly to the attributes, a dictionary of datasets can be obtained using the `items` method and dictionary comprehension.

In [65]:
datasets = dict(mesh.items())
datasets = {item:np.array(datasets[item]) for item in datasets}
print(datasets)

{'cellFace': array([     0,      1,      2, ..., 382717, 382718, 382719], dtype=int32), 'faceBC': array([[9],
       [0],
       [0],
       ...,
       [9],
       [9],
       [9]], dtype=int32), 'faceCell': array([[     0, 124800],
       [     0,      1],
       [     0,  88331],
       ...,
       [124799, 141437],
       [124799, 141438],
       [124799, 141439]], dtype=int32), 'faceInfo': array([[2, 0],
       [0, 0],
       [0, 0],
       ...,
       [2, 0],
       [2, 0],
       [2, 0]], dtype=int32), 'faceNodes': array([[    13],
       [   286],
       [   273],
       ...,
       [133204],
       [133192],
       [133191]], dtype=int32), 'faceType': array([[4],
       [4],
       [4],
       ...,
       [4],
       [4],
       [4]], dtype=int32), 'nodeVertex': array([[ 3.31865544e+02,  4.49082846e-03,  0.00000000e+00],
       [ 2.33772516e+02,  3.93497335e-03, -2.82267962e+00],
       [ 1.50669013e+02,  4.49082846e-03, -5.12891315e+00],
       ...,
       [ 2.38361856e+02,  

Going through these logically then:

## nodeVertex

`[numNodes x 3]` array

where `numNodes` is the total number of unique nodes in the mesh. Each row is an `[x,y,z]` coordinate for a node location.

In [66]:
numNodes = datasets['nodeVertex'].shape[0]

print('numNodes = {}'.format(numNodes))
print('nodeVertex:')
print(datasets['nodeVertex'])

numNodes = 133205
nodeVertex:
[[ 3.31865544e+02  4.49082846e-03  0.00000000e+00]
 [ 2.33772516e+02  3.93497335e-03 -2.82267962e+00]
 [ 1.50669013e+02  4.49082846e-03 -5.12891315e+00]
 ...
 [ 2.38361856e+02  2.90701659e+02  2.96788236e+02]
 [ 2.93968040e+02  2.90701659e+02  2.96788236e+02]
 [ 3.72019762e+02  2.90701659e+02  2.96788236e+02]]


## faceType

`[numFaces x 1]` array

The number of nodes making up each face. `Face0` has `faceType[0]` nodes etc...



In [67]:
print('faceType shape = {}'.format(datasets['faceType'].shape))
print('faceType:')
print(datasets['faceType'])

faceType shape = (382720, 1)
faceType:
[[4]
 [4]
 [4]
 ...
 [4]
 [4]
 [4]]


## faceNodes

`[sum(faceType) x 1]` array

Ordered list of nodes making up each face. The order is determined by the commulative summation of the `faceType` dataset. For indexing indvidual faces it can be useful to define an additional data type of `faceIndex`, where each entry is the sum of `faceType` up to the particular index, then the nodes in face n are `faceNodes[faceIndex[n]:faceIndex[n]+faceType[n]]`.




In [68]:
print('sum(faceType) = {}'.format(np.sum(datasets['faceType'])))
print('faceNodes shape = {}'.format(datasets['faceNodes'].shape))

print('faceNodes:')
print(datasets['faceNodes'])

# Create faceIndex array
faceIndex = np.zeros_like(datasets['faceType'])

for i in range(attributes['numFaces']-1):
    faceIndex[i+1] = datasets['faceType'][i+1] + faceIndex[i]

print('faceIndex:')
print(faceIndex)

sum(faceType) = 1530880
faceNodes shape = (1530880, 1)
faceNodes:
[[    13]
 [   286]
 [   273]
 ...
 [133204]
 [133192]
 [133191]]
faceIndex:
[[      0]
 [      4]
 [      8]
 ...
 [1530868]
 [1530872]
 [1530876]]


## Example: find the nodes making up a random face from the mesh

In [69]:
# Random face index
faceID = np.random.randint(0,high=attributes['numFaces'])
n_nodes = int(datasets['faceType'][faceID])

print('faceID: {}'.format(faceID))
print('nodes in face: {}'.format(n_nodes))

for i in range(n_nodes):
    print('node: {}, id: {}, [x,y,z]: {}'.format(i+1,int(datasets['faceNodes'][faceIndex[faceID]+i]),datasets['nodeVertex'][datasets['faceNodes'][faceIndex[faceID]+i]]))


faceID: 312621
nodes in face: 4
node: 1, id: 109421, [x,y,z]: [[[ 52.09053259  44.24420207 -10.13687813]]]
node: 2, id: 109422, [x,y,z]: [[[51.62639787 44.47465977 -9.99349056]]]
node: 3, id: 109409, [x,y,z]: [[[51.84805289 43.45316597 -7.0328313 ]]]
node: 4, id: 109408, [x,y,z]: [[[52.23318846 43.25625616 -7.13768575]]]


---
## faceCell

`[numFaces x 2]` array

Pairs of left and right cell indices for each face in the mesh. Boundary faces should have the interior cell in the left index, with a unique, *consequtively* numbered boundary cell on the right.

In [70]:
print('faceCell shape = {}'.format(datasets['faceCell'].shape))
print('faceCell:')
print(datasets['faceCell'])

faceCell shape = (382720, 2)
faceCell:
[[     0 124800]
 [     0      1]
 [     0  88331]
 ...
 [124799 141437]
 [124799 141438]
 [124799 141439]]


## faceBC

`[numFaces x 1]` array

The boundary condition to be applied to each face. Numbering here follows the fluent numbering convention:
* 0 = NONE
* 2 = Interior
* 3 = Wall
* 4 = Inflow
* 5 = Outflow
* 7 = Symmetry
* 9 = Farfield
* 12 = Periodic (Additional zone info needed)
* 13 = Wall source (Accoustic solver only)

In [71]:
print('faceBC shape: {}'.format(datasets['faceBC'].shape))
print('faceBC:')
print(datasets['faceBC'])

faceBC shape: (382720, 1)
faceBC:
[[9]
 [0]
 [0]
 ...
 [9]
 [9]
 [9]]


## faceInfo

`[numFaces x 2]` array

Zone information for each face used to initialise conditions from fluidZones. The first index should be zoneID, and the second should be 0. This is used for defining more complex boundary conditions- for example periodic faces, or multiple types of wall (slip vs non-slip) etc...

If used for boundary conditions, these indexes should correspond to entries in the control dictionary.

In [72]:
print('faceInfo shape: {}'.format(datasets['faceInfo'].shape))
print('faceInfo:')
print(datasets['faceInfo'])

faceInfo shape: (382720, 2)
faceInfo:
[[2 0]
 [0 0]
 [0 0]
 ...
 [2 0]
 [2 0]
 [2 0]]


## cellZone (optional)

`[numCells x 1]` array

Contains index for what mesh zone the cell is in. Becomes important for MRF (floating rotor) type simulations, where some cells will be in a rotating reference frame, and other stationary.



## Bash commands
---

There are a number of bash commands which can be useful for quick analysis of hdf5 files. These include:

### `h5ls` 

```
> h5ls MDO_125K.h5
mesh                     Group
```
In particular h5ls -r list the group structure recursively
```
> h5ls -r MDO_125K.h5
/                        Group
/mesh                    Group
/mesh/faceBC             Dataset {382720, 1}
/mesh/faceCell           Dataset {382720, 2}
/mesh/faceInfo           Dataset {382720, 2}
/mesh/faceNodes          Dataset {1530880, 1}
/mesh/faceType           Dataset {382720, 1}
/mesh/nodeVertex         Dataset {133205, 3}
```

* `h5dump`